# Add Reviewers to Database

### January 2014 Neil D. Lawrence
#### Updated October 2014, onvert to new cmtutils May 2021

This notebook allows us to merge in reviewers from a CMT export file. It is to be used when you aren't confident about whether the reviewers being merged in are already in the data base or not. For this reason it will query you more carefully over whether a reviewer should be added.

### Backing Up
The first thing is to load in the reviewer data base. I suggest you **back up** your data base before doing this operation just in case something goes wrong.

In [1]:
import cmtutils as cu

In [2]:
db = cu.Reviewerdb('test.db')

## Adding Reviewers from CMT Export

Now we specify the source of (potential) reviewers we are loading in. In this notebook we consider two examples where we are loading in new data base users. In the first example the source could be the previous list of reviewers from the conference (found via a CMT export).

In [5]:
# this file can be a CMT user list exported
filename='aistats2012reviewers.xls'
# this field will tag the data base for the source of the user.
fieldname = 'FromAISTATS2012' 
new_users=cu.CMT_Reviewers_read(filename=filename, header=2, dataframe=True)

ValueError: Data has no column Email for index.

In [ ]:
%debug

> /Users/neil/opt/anaconda3/lib/python3.8/site-packages/cmtutils-0.1.0-py3.8.egg/cmtutils/cmtutils.py(1377)__init__()
   1375                                 raise ValueError("Data frame needs an index column.")
   1376                             if not index_col in item.keys():
-> 1377                                 raise ValueError("Data has no column " + index_col + " for index.")
   1378                             index_val = item[index_col]
   1379                             del item[index_col]

ipdb> item
{}


Now we can add in the field for recording the reviewer source to the data base.

In [4]:
db.add_field(fieldname, 'INTEGER') # add the recording field if it's not already present

# This line will set the field name to NULL if required. Turned off by default.
if False:
    db.update_fields(db.ids(), fieldname=fieldname, fieldvalue=None)

If we want to see the list of new reviewers we can check who they are. They are stored as a `pandas` data frame.

In [5]:
new_users.reviewers

### Adding Reviewers to the Data Base

Try and match or add the reviewer to the system. You don't want to necessarily add them as reviewers straight away, but you may need to know the origin of the new users in the data base. For that reason we add the `user_list` field name to the data base and set its value to 1 if any users are added to the data base from this source.

You will be queried for each user whether you want them to be added to the local data base. 

In [5]:
db.add_users(new_users.reviewers, yes=False, query=True, fieldname=fieldname)

AttributeError: 'list' object has no attribute 'iterrows'

In [ ]:
%debug

> /Users/neil/opt/anaconda3/lib/python3.8/site-packages/cmtutils-0.1.0-py3.8.egg/cmtutils/cmtutils.py(1511)add_users()
   1509     def add_users(self, reviewers, fieldname=None, yes=False, query=True, match_firstname=False, match_lastname=True):
   1510         count = 0
-> 1511         for i, reviewer in reviewers.iterrows():
   1512             print("Processed ", count, " out of ", len(reviewers), " reviewers.")
   1513             id = self.match_or_add_reviewer(reviewer, yes=yes, query=query, match_firstname=match_firstname, match_lastname=match_lastname, fieldname=fieldname)

ipdb> reviewers
[{'FirstName': 'Andrew', 'MiddleNames': ' ', 'LastName': 'Golightly', 'Institute': 'University of Newcastle', 'Email': 'a.golightly@ncl.ac.uk'}, {'FirstName': 'Ata', 'MiddleNames': ' ', 'LastName': 'Kaban', 'Institute': None, 'Email': 'a.kaban@cs.bham.ac.uk'}, {'FirstName': 'Adam', 'MiddleNames': 'M', 'LastName': 'Johansen', 'Institute': 'Warwick', 'Email': 'a.m.johansen@warwick.ac.uk'}, {'Fi

Now you have finished the queries you can check how many users you've added with the following SQL query on the data base.

In [ ]:
# This is number of added users.
db._execute_sql('SELECT COUNT(Active) FROM Reviewers WHERE ' + fieldname + '=1;')

### Exporting Reviewers

Now we can export the reviewers ready for importing into CMT.

In [ ]:
db.export_reviewers(filename='aistats_2012_reviewers.txt', sql=fieldname + '=1')

## Incorporating Google Sheet Information

For NIPS 2014 we had our area chairs suggest reviewers in a google spreadsheet. We used a form for this. The form created reviewer suggestions with the fields we need for CMT (We also stored our own suggestions for reviewers in a google spreadsheet. Below we show how to extract informatinon from the google spreadsheet (the spreadsheet key is set in the `defaults.cfg` file or your own `.cmt_user.cfg` configuration file). 

In [ ]:
import pods
pods.google.api_available

In [ ]:
from cmtutils import config
spreadsheet_key = config.config.get('google docs', 'reviewer_candidate_key')
spreadsheet_name = 'Area Chair Reviewer Suggestions'
fieldname = 'AreaChairSuggestion'
# Drive_store currently not working.
ds = cu.drive_store(spreadsheet_key, spreadsheet_name) # this opens the google sheet.
# Column_fields causes these new column names to replace those in the spreadsheet 
# (which come from the form) in the pandas DataFrame we recover
column_fields={'1':'TimeStamp', '2':'FirstName', '3':'LastName', '4':'MiddleNames', 
               '5':'Email', '6':'Institute', '7':'Nominator', '9':'ScholarID'}
ds.read(column_fields=column_fields)

Then if you want to add them to the data base you can use the `add_users()` feature as before.

In [ ]:
db.add_field(fieldname, 'INTEGER') # add the recording field if it's not already present
db.add_users(ds.reviewers, yes=False, query=True, fieldname=fieldname)

Now we can export a tab separated values file ready for import into CMT.

In [ ]:
db.export_reviewers(filename='area_chair_suggestions.txt', sql=fieldname +'=1')